In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

c:\Users\PabloAlejandroGuatib\Pablo\personal\repositorios\python-whatsapp-bot\chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
INPUT_DIR = "../../../graphrag_reduced/output"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

In [3]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="default-entity-description",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 628


,id,human_readable_id,title,community,level,degree,x,y
0,a7bcdb89-1d9a-410a-9324-5b1e5e290c26,0,AUDI S3 SEDÁN TFSI QUATTRO,-1,0,1,0.0,0.0
1,a17ba9eb-6888-451b-af97-a7412ea3b24f,1,PRECIO INICIAL: $249.900.000,-1,0,1,0.0,0.0
2,66bc9702-3cb9-463e-b3d7-90a7d0f10b94,2,CUATRO CILINDROS EN LÍNEA,-1,0,3,0.0,0.0
3,407ccaaf-b5f2-4789-afc0-49291217a6d9,3,S-TRONIC DE 7 CAMBIOS,-1,0,1,0.0,0.0
4,4e264cbe-3b2c-4a7a-8074-8ef834a0e0ab,4,ACELERACIÓN (0 A 100 KM/H): 5.0 SEGUNDOS,0,0,1,0.0,0.0


In [4]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()


Relationship count: 256


,id,human_readable_id,source,target,description,weight,combined_degree,text_unit_ids
0,063601f0-60fd-4691-ba54-d0055b3e6d9e,0,AUDI S3 SEDÁN TFSI QUATTRO,PRECIO INICIAL: $249.900.000,El precio inicial se refiere al costo del mode...,8.0,2,[17a36093436dba59f919711728dfbc006ee25724db974...
1,727e95f9-c5e5-49e4-a076-df8565ca58b4,1,CUATRO CILINDROS EN LÍNEA,S-TRONIC DE 7 CAMBIOS,El motor de cuatro cilindros en línea está aco...,9.0,4,[17a36093436dba59f919711728dfbc006ee25724db974...
2,fee5e1de-8213-4d5f-bcf5-f017a70c7d25,2,ACELERACIÓN (0 A 100 KM/H): 5.0 SEGUNDOS,VELOCIDAD MÁXIMA: 250 KM/H,Ambos son indicadores de desempeño del vehículo,10.0,7,[17a36093436dba59f919711728dfbc006ee25724db974...
3,faf19269-889b-4205-9034-bce8622f8999,3,LONGITUD: 4.504 MM,ANCHO CON ESPEJOS: 1.984 MM,Ambos son dimensiones físicas del vehículo,7.0,5,[17a36093436dba59f919711728dfbc006ee25724db974...
4,b50bb45e-7d96-4ad0-9ebb-3f2d9a122aa4,4,SUSPENSIÓN DELANTERA: BRAZO TELESCÓPICO MCPHERSON,SUSPENSIÓN TRASERA: MULTILINK DE 4 BRAZOS,La suspensión del Audi RS Q3 incluye un sistem...,37.0,6,[17a36093436dba59f919711728dfbc006ee25724db974...


In [5]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 14


,id,human_readable_id,community,parent,level,title,summary,full_content,rank,rank_explanation,findings,full_content_json,period,size
0,02bd3f77b1ee46a28dae6b91a86ac230,8,8,1,1,Audi Q8 55 TFSI Quattro MHEV Progressive,La comunidad se centra en el modelo Audi Q8 55...,# Audi Q8 55 TFSI Quattro MHEV Progressive\n\n...,8.5,La calificación es alta debido a la relevancia...,[{'explanation': 'El Audi Q8 55 TFSI Quattro M...,"{\n ""title"": ""Audi Q8 55 TFSI Quattro MHEV ...",2025-02-23,9
1,5d3c92cd02d7428e94899f175025a8da,9,9,1,1,Audi Q8 55 TFSI quattro MHEV y su Tecnología A...,La comunidad se centra en el Audi Q8 55 TFSI q...,# Audi Q8 55 TFSI quattro MHEV y su Tecnología...,8.5,La calificación es alta debido a la relevancia...,[{'explanation': 'El motor de 2.995 cc del Aud...,"{\n ""title"": ""Audi Q8 55 TFSI quattro MHEV ...",2025-02-23,2
2,7ae2fba39f8e4b53ba83ea95f3c6367e,10,10,1,1,Audi Q8 55 TFSI quattro MHEV y Especificacione...,La comunidad se centra en el Audi Q8 55 TFSI q...,# Audi Q8 55 TFSI quattro MHEV y Especificacio...,8.5,La calificación es alta debido a la relevancia...,[{'explanation': 'El peso en vacío del Audi Q8...,"{\n ""title"": ""Audi Q8 55 TFSI quattro MHEV ...",2025-02-23,2
3,be7a912c99de4e3a97270bf5aad35fc2,11,11,2,1,Audi RS3 Sportback y sus Especificaciones,La comunidad se centra en el Audi RS3 Sportbac...,# Audi RS3 Sportback y sus Especificaciones\n\...,8.5,La calificación es alta debido a la relevancia...,[{'explanation': 'El Audi RS3 Sportback se dis...,"{\n ""title"": ""Audi RS3 Sportback y sus Espe...",2025-02-23,7
4,7bc7d1bd198f4593839384961fca4853,12,12,2,1,Audi Q8 55 TFSI quattro MHEV y su Dinámica de ...,La comunidad se centra en el Audi Q8 55 TFSI q...,# Audi Q8 55 TFSI quattro MHEV y su Dinámica d...,8.5,La calificación es alta debido a la relevancia...,[{'explanation': 'La suspensión delantera del ...,"{\n ""title"": ""Audi Q8 55 TFSI quattro MHEV ...",2025-02-23,4


In [6]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 29


,id,human_readable_id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,17a36093436dba59f919711728dfbc006ee25724db9748...,1,Audi S3 Sedán TFSI quattro - Ficha Técnica Com...,949,[0176fe4f3cb16e417b15b36f7797b9e0e16dbe4d25230...,"[a7bcdb89-1d9a-410a-9324-5b1e5e290c26, a17ba9e...","[063601f0-60fd-4691-ba54-d0055b3e6d9e, 727e95f..."
1,53fbf6c77a74e72173b94b3193ad7fd8eabd00d0a3f624...,2,Audi RS3 Sportback - Ficha Técnica Completa - ...,1076,[08c17ef4280f76430be642cfe6f21c49f6bc2585abb84...,"[8d32b0a8-0784-430d-833d-d0760653c298, a1eba80...","[83c236b7-50d2-42ce-8d80-0cc159be7c45, edb5969..."
2,e2f5289b98a949a0ec3c10b465a6b397625cdbe74c479f...,3,Audi Q3 Sportback 35 TFSI Ambition Plus - Fich...,1143,[12ebcd4306a0e57db19da9932457320cb891d07fc5edf...,"[66bc9702-3cb9-463e-b3d7-90a7d0f10b94, 2884496...","[697673f4-93f0-4e94-9c00-969a0bcd1b79, 9c4da85..."
3,b1a39ca9289244cc73f087b8fea855b9c8dee71e9461a8...,4,ámico de accionamiento eléctrico\n- Apoya braz...,43,[12ebcd4306a0e57db19da9932457320cb891d07fc5edf...,"[f1767435-fb25-4831-8116-23cb5506399e, 8b7b2d1...","[9324a574-e677-4037-92fe-e34cc7ec3fe1, 724f2dd..."
4,02ec64d44ac9d2832dc81d0724cafee20a68be595587bc...,5,Audi Q7 55 TFSI quattro MHEV Prestige - Ficha ...,1200,[1f38568f11c47995c3f238adbadeb3d1752dfde264174...,"[8d32b0a8-0784-430d-833d-d0760653c298, ac6bbb5...","[0227e6c9-a5df-440d-8717-6e7cdacb3023, 6f8209d..."


In [7]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

In [8]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

In [9]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [10]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [ ]:
result = await search_engine.asearch("Carros electricos")
print(result.response)

En el mercado actual, los vehículos eléctricos están ganando popularidad debido a su eficiencia energética y menor impacto ambiental. Sin embargo, encontrar opciones por debajo de los 200 millones de pesos puede ser un desafío, ya que muchos modelos eléctricos tienden a ser más costosos debido a la tecnología avanzada que incorporan.

### Opciones Disponibles

En los datos proporcionados, no se menciona ningún vehículo eléctrico con un precio inicial por debajo de los 200 millones de pesos. Los modelos listados, como el Volvo EX30 Core Ultra E60 2024, tienen un precio inicial de $259.990.000 [Data: Sources (16)]. Otros modelos, como el Audi A4 40 TFSI MHEV Advanced, que es un vehículo híbrido, tienen un precio de $179.900.000, pero no es completamente eléctrico [Data: Entities (472); Relationships (194)].

### Consideraciones Generales

Para aquellos interesados en vehículos eléctricos a un precio más accesible, es posible que deban considerar opciones de segunda mano o esperar a que n